In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/steam-reviews/output_steamspy.csv
/kaggle/input/steam-reviews/output.csv


In [2]:
import os
import tqdm
import collections
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
import torch
import torch.nn as nn
import transformers
import matplotlib.pyplot as plt

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("filipkin/steam-reviews")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/steam-reviews


In [5]:
df=pd.read_csv('/kaggle/input/steam-reviews/output.csv')

In [6]:
df.head(10)

,id,app_id,content,author_id,is_positive
0,181331361,100,At least its a counter strike -1/100,76561199556485100,Negative
1,180872601,100,Uh... So far my playthrough has not been great...,76561199230620391,Negative
2,177836246,100,Better mechanics than cs2,76561198417690647,Negative
3,177287444,100,buggy mess and NOT fun to play at all,76561199077268730,Negative
4,176678990,100,"Whoever came up with this, is gonna fucking ge...",76561199104544266,Negative
5,176118004,100,Every enemy and their grandmother have aimbot ...,76561198296823531,Negative
6,172896379,100,l,76561199182060159,Negative
7,171821329,100,Counter Strike: Condition Zero Deleted Scenes(...,76561199032983213,Negative
8,171639855,100,[REVIEW IS FOR DELETED SCENES] Game is pretty ...,76561198866570651,Negative
9,167078851,100,too many enemies,76561199554114911,Negative


In [7]:
#need the content,label that's all so im going to take content and is_positive thats it

In [8]:
df.isnull().sum()

id               0
app_id           0
content        428
author_id        0
is_positive      0
dtype: int64

In [9]:
df.isnull().sum()

id               0
app_id           0
content        428
author_id        0
is_positive      0
dtype: int64

In [10]:
df=df.dropna()

In [11]:
df.isnull().sum()

id             0
app_id         0
content        0
author_id      0
is_positive    0
dtype: int64

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
stop_words=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()

def preprocess_text(text):
    text=re.sub(r'<.*?>','',text)
    text=text.lower()
    text=re.sub(r'[^a-z\s]','',text)
    tokens=word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [14]:
df['content']=df.content.apply(preprocess_text)

In [15]:
content=df['content']

In [16]:
label=df['is_positive']

In [17]:
print(content[:10])

0                                 least counter strike
1    uh far playthrough great glitched texture cont...
2                                    better mechanic c
3                                  buggy mess fun play
4    whoever came gon na fucking get negative revie...
5    every enemy grandmother aimbot even funny died...
6                                                    l
7    counter strike condition zero deleted scene c ...
8    review deleted scene game pretty fun play like...
9                                           many enemy
Name: content, dtype: object


In [18]:
print(label[:30])

0     Negative
1     Negative
2     Negative
3     Negative
4     Negative
5     Negative
6     Negative
7     Negative
8     Negative
9     Negative
10    Negative
11    Negative
12    Negative
13    Negative
14    Negative
15    Negative
16    Negative
17    Negative
18    Negative
19    Negative
20    Negative
21    Negative
22    Negative
23    Negative
24    Negative
25    Negative
26    Negative
27    Negative
28    Negative
29    Negative
Name: is_positive, dtype: object


In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
le=LabelEncoder()
label=le.fit_transform(label)

In [21]:
print(label[:10])

[0 0 0 0 0 0 0 0 0 0]


In [22]:
#O:NEGATIVE, 1 : POSITIVE

In [23]:
content=np.asarray(content)

In [24]:
label=np.asarray(label)

In [25]:
print(label)

[0 0 0 ... 0 0 0]


# Create a dataset
1. Define a tokenizer
2. Make tokens
3. get vocab
4. get itos,stoi
5. Create a dataset class
6. Create a dataloader

In [26]:
seed = 1234

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [27]:
t_name='bert-base-uncased'
tokenizer=transformers.AutoTokenizer.from_pretrained(t_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [28]:
tokenizer.tokenize('Elden ring has to be the greatest game ever made')

['el',
 '##den',
 'ring',
 'has',
 'to',
 'be',
 'the',
 'greatest',
 'game',
 'ever',
 'made']

In [29]:
content.shape,label.shape

((200723,), (200723,))

In [30]:
def tokenize(data,tokenizer):
    encoded=tokenizer(data,
        add_special_tokens=True,
        max_length=128,
        truncation=True,
        padding='max_length',
                     )
    return encoded['input_ids']

In [31]:
# Tokenize all at once and unpack
ids=[tokenize(content[i],tokenizer) for i in range(len(content))]

In [32]:
print(ids[:1])

[[101, 2560, 4675, 4894, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [33]:
tokenizer.vocab['balls']

7395

In [34]:
tokenizer.pad_token

'[PAD]'

In [35]:
tokenizer.pad_token_id

0

In [36]:
tokenizer.vocab[tokenizer.pad_token]

0

In [37]:
pad_index = tokenizer.pad_token_id

In [38]:
print(pad_index)

0


In [39]:
print(type(ids))

<class 'list'>


In [40]:
print(type(label))

<class 'numpy.ndarray'>


In [41]:
from sklearn.model_selection import train_test_split

In [42]:
X_train, X_temp, y_train, y_temp = train_test_split(ids, label, test_size=0.2, stratify=label, random_state=42)

In [43]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [44]:
from torch.utils.data import Dataset,DataLoader

In [45]:
class Reviews(Dataset):
    def __init__(self,X,y):
        super().__init__()
        self.X=X
        self.y=y

    def __len__(self):
        return len(self.X)

    def __getitem__(self,id):
        return self.X[id],self.y[id]

In [46]:
len(X_train),len(y_train)

(160578, 160578)

In [47]:
len(X_test),len(y_test)

(20073, 20073)

In [48]:
len(X_val),len(y_val)

(20072, 20072)

In [51]:
##X_train_tensor = torch.stack(X_train).squeeze(1)
##print(f"X_train_tensor shape: {X_train_tensor.shape}")  # Should be [160578, 128]

In [52]:
##X_val_tensor = torch.stack(X_val).squeeze(1)
##print(f"X_train_tensor shape: {X_val_tensor.shape}")  # Should be [160578, 128]

In [53]:
##X_train_tensor = torch.stack(X_val).squeeze(1)
##print(f"X_train_tensor shape: {X_val_tensor.shape}")  # Should be [160578, 128]

In [54]:
X_train_tensor = torch.tensor(X_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)

In [55]:
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [56]:
print(f"X_train_tensor shape: {X_train_tensor.shape}")
print(f"X_val_tensor shape: {X_val_tensor.shape}")
print(f"X_test_tensor shape: {X_test_tensor.shape}")

X_train_tensor shape: torch.Size([160578, 128])
X_val_tensor shape: torch.Size([20072, 128])
X_test_tensor shape: torch.Size([20073, 128])


In [57]:
print(f"y_train_tensor shape: {y_train_tensor.shape}")
print(f"y_val_tensor shape: {y_val_tensor.shape}")
print(f"y_test_tensor shape: {y_test_tensor.shape}")

y_train_tensor shape: torch.Size([160578])
y_val_tensor shape: torch.Size([20072])
y_test_tensor shape: torch.Size([20073])


In [58]:
train_data=Reviews(X_train_tensor,y_train_tensor)

In [59]:
test_data=Reviews(X_test_tensor,y_test_tensor)

In [60]:
val_data=Reviews(X_val_tensor,y_val_tensor)

In [61]:
train_dataloader=DataLoader(train_data,shuffle=True,batch_size=16)

In [62]:
val_dataloader=DataLoader(val_data,batch_size=16)

In [63]:
for batch_idx, (input_ids, labels) in enumerate(train_dataloader):
    print(f"Batch {batch_idx}:")
    print(f"Input IDs shape: {input_ids.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Input IDs dtype: {input_ids.dtype}")
    print(f"Labels dtype: {labels.dtype}")
    print(f"First sample input IDs: {input_ids[0][:10]}...")  # First 10 tokens
    print(f"First sample label: {labels[0]}")
    break  # Only check first batch

Batch 0:
Input IDs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Input IDs dtype: torch.int64
Labels dtype: torch.int64
First sample input IDs: tensor([  101, 10514, 18098,  9355,  2135,  3835,   102,     0,     0,     0])...
First sample label: 1


# Make a transformer model


In [65]:
class Transformer(nn.Module):
    def __init__(self,transformer,output_dim,freeze):
        super().__init__()
        self.transformer=transformer
        hidden_dim=transformer.config.hidden_size
        self.fc=nn.Linear(hidden_dim,output_dim)
        if freeze:
            for param in self.transformer.parameters():
                param.requires_grad=False

    def forward(self,ids):
        attention_mask = (ids != 0).long()
        output=self.transformer(ids,attention_mask,output_attentions=True)
        hidden=output.last_hidden_state
        attention=output.attentions[-1]
        cls_hidden=hidden[:,0,:]
        preds=self.fc(torch.tanh(cls_hidden))
        return preds

In [66]:
transformer = transformers.AutoModel.from_pretrained(t_name)

2025-07-29 06:43:23.734556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753771403.942048      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753771404.003968      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [67]:
transformer.config.hidden_size

768

In [68]:
output_dim=len(torch.unique(y_train_tensor))
freeze=False
model=Transformer(transformer,output_dim,freeze)

In [69]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 109,483,778 trainable parameters


In [70]:
import torch.optim as optim

In [71]:
optim=optim.Adam(params=model.parameters(),lr=2e-5)

In [72]:
loss_fn=nn.CrossEntropyLoss()

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [74]:
model=model.to(device)
loss_fn=loss_fn.to(device)

In [75]:
def acc_fn(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy

In [76]:
def train(model,dataloader,loss_fn,optim,acc_fn,device):
    model.train()
    train_loss=[]
    train_acc=[]
    for batch,(X,y) in tqdm.tqdm(enumerate(dataloader)):
        X=X.to(device)
        y=y.to(device)
        pred=model(X)
        loss=loss_fn(pred,y)
        train_loss.append(loss.item())
        acc=acc_fn(pred,y)
        train_acc.append(acc.item())
        optim.zero_grad()
        loss.backward()
        optim.step()
    return np.mean(train_loss),np.mean(train_acc)

In [77]:
def test(model,dataloader,loss_fn,acc_fn,device):
    model.eval()
    test_loss=[]
    test_acc=[]
    with torch.inference_mode():
        for batch,(X,y) in tqdm.tqdm(enumerate(dataloader)):
            X=X.to(device)
            y=y.to(device)
            pred=model(X)
            loss=loss_fn(pred,y)
            test_loss.append(loss.item())
            acc=acc_fn(pred,y)
            test_acc.append(acc.item())
    return np.mean(test_loss),np.mean(test_acc)

In [78]:
epochs=3
best_valid_loss=float("inf")

metrics = collections.defaultdict(list)

for epoch in range(epochs):
    train_loss,train_acc=train(model,train_dataloader,loss_fn,optim,acc_fn,device)
    valid_loss,valid_acc=test(model,val_dataloader,loss_fn,acc_fn,device)
    metrics["train_losses"].append(train_loss)
    metrics["train_accs"].append(train_acc)
    metrics["valid_losses"].append(valid_loss)
    metrics["valid_accs"].append(valid_acc)
    if valid_loss<best_valid_loss:
        best_valid_loss=valid_loss
        torch.save(model.state_dict(), "transformer.pt")
    print(f"epoch: {epoch}")
    print(f"train_loss: {train_loss:.3f}, train_acc: {train_acc:.3f}")
    print(f"valid_loss: {valid_loss:.3f}, valid_acc: {valid_acc:.3f}")

0it [00:00, ?it/s]BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
10037it [31:46,  5.26it/s]
1255it [01:11, 17.60it/s]


epoch: 0
train_loss: 0.362, train_acc: 0.833
valid_loss: 0.339, valid_acc: 0.846


10037it [31:47,  5.26it/s]
1255it [01:11, 17.60it/s]


epoch: 1
train_loss: 0.285, train_acc: 0.874
valid_loss: 0.336, valid_acc: 0.852


10037it [31:47,  5.26it/s]
1255it [01:11, 17.60it/s]

epoch: 2
train_loss: 0.223, train_acc: 0.904
valid_loss: 0.360, valid_acc: 0.849


In [82]:
torch.save('model.state_dict()','/kaggle/working/steamtrans.pt')